In [1]:
import torch
import torch.nn as nn

import tensorflow as tf
import numpy as np

from tqdm import tqdm
from nobuco import pytorch_to_keras, ChannelOrder

from model.TwinLiteRELU import TwinLiteNet as TwinLiteNetRELU 
from DataSet import CustomDataset

2024-08-20 10:41:37.361460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 10:41:37.361527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 10:41:37.387133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 10:41:37.449929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 10:41:38.622988: W tensorflow/compiler/tf2

In [2]:
state_dict = torch.load('pretrained/epo64_RELU.pth', map_location='cpu')

model = nn.Sequential(torch.ao.quantization.QuantStub(), TwinLiteNetRELU(), torch.ao.quantization.DeQuantStub())
model.load_state_dict(state_dict)
model = model[1]
model.eval()

/home/ubuntadmin/Desktop/TwinLiteNet/venv/lib/python3.11/site-packages/nobuco/trace/trace.py:366: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  outputs = orig_method(*args, 

TwinLiteNet(
  (encoder): ESPNet_Encoder(
    (level1): CBR(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (sample1): InputProjectionA(
      (pool): ModuleList(
        (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (sample2): InputProjectionA(
      (pool): ModuleList(
        (0-1): 2 x AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (b1): CBR(
      (conv): Conv2d(19, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(19, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (level2_0): DownSamplerB(
      (c1): C(
        (conv): Conv2d(19, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (d1): CDilated(
        (conv): Conv2d(12, 16, kernel_size

In [3]:
DATASET_PATH = "/media/ubuntadmin/67b3a250-6e0f-4a86-b15d-1b302250e0b4/home/ubuntadmin/Desktop/data/datasets/road-segmentation/TwinLiteTree/images"
MAX_CALIBRATION_IMG = 1
dataset = CustomDataset(DATASET_PATH, valid=True)
dataset.names = dataset.names[:MAX_CALIBRATION_IMG]

#HEIGHT_FACTOR = 1.0
HEIGHT_FACTOR = 2.0 / 3.0

In [4]:
input = torch.randn(1, 3, int(HEIGHT_FACTOR * dataset.H_), dataset.W_)
keras_model = pytorch_to_keras(model, [input], inputs_channel_order=ChannelOrder.TENSORFLOW)

[Nobuco] Tracing: 0 ops [00:00]

[Nobuco] Tracing (DONE): 236 ops [00:02]
[Nobuco] Converting: |▏         | 4/162 ops [00:03]

2024-08-20 10:41:56.293248: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[Nobuco] Converting: |██████▌   | 106/162 ops [00:05]

2024-08-20 10:41:59.215396: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23040000 exceeds 10% of free system memory.


[Nobuco] Converting: |███████   | 114/162 ops [00:06]

2024-08-20 10:41:59.457647: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23040000 exceeds 10% of free system memory.
2024-08-20 10:41:59.461522: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23040000 exceeds 10% of free system memory.
2024-08-20 10:41:59.483652: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23040000 exceeds 10% of free system memory.
2024-08-20 10:41:59.490354: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23040000 exceeds 10% of free system memory.


[Nobuco] Converting (DONE): |██████████| 162/162 ops [00:08]
Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

TwinLiteNet[model.TwinLiteRELU](float32_0<1,3,240,640>) -> (float32_265<1,2,240,640>, float32_286<1,2,240,640>)
 │  ESPNet_Encoder[model.TwinLiteRELU](float32_0<1,3,240,640>) -> float32_244<1,32,30,80>
 │   │  CBR[model.TwinLiteRELU](float32_0<1,3,240,640>) -> float32_7<1,16,120,320>
 │   │   │  Conv2d[torch.nn.modules.conv](float32_0<1,3,240,640>) -> float32_2<1,16,120,320>
 │   │   │   └· conv2d[torch.nn.functional](float32_0<1,3,240,640>, float32_1<16,3,3,3>, None, (2, 2), (1, 1), (1, 1), 1) -> float32_2<1,16,120,320>
 │   │   │  BatchNorm2d[torch.nn.modules.batchnorm](

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = converter.convert()
with open(f"models/twinlitenet.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmprewvbcyt/assets


INFO:tensorflow:Assets written to: /tmp/tmprewvbcyt/assets
2024-08-20 10:42:20.864936: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-20 10:42:20.864980: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-20 10:42:20.866320: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprewvbcyt
2024-08-20 10:42:20.892758: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-20 10:42:20.892789: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmprewvbcyt
2024-08-20 10:42:20.941032: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-08-20 10:42:20.960809: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-20 10:42:21.282074: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [6]:
def representative_dataset_gen():
	for img_name, input, (seg_da, seg_ll) in tqdm(dataset):
		input = input.float() / 255.0
		input = input.numpy()
		input = np.expand_dims(input, axis=0)
		input = np.transpose(input, (0, 2, 3, 1))

		input = input[:, int((1 - HEIGHT_FACTOR) * dataset.H_) :, :, :]

		yield [input]

# DOES NOT WORK
def random_representation_gen():
	# USE IMAGENET MEAN AND STD
	input_transform_mean=[0.485, 0.456, 0.406]
	input_transform_std=[0.229, 0.224, 0.225]

	input = np.random.rand(1, dataset.H_, dataset.W_, 3)
	input = input * input_transform_std + input_transform_mean
	yield [input.astype(np.float32)]

qt_converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
qt_converter.optimizations = [tf.lite.Optimize.DEFAULT]
qt_converter.representative_dataset = representative_dataset_gen
#qt_converter.representative_dataset = random_representation_gen

tflite_model = qt_converter.convert()
with open(f"models/twinlitenet_qt.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp44c2wt1k/assets


INFO:tensorflow:Assets written to: /tmp/tmp44c2wt1k/assets
/home/ubuntadmin/Desktop/TwinLiteNet/venv/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-08-20 10:42:40.402823: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-20 10:42:40.402851: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-20 10:42:40.403029: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp44c2wt1k
2024-08-20 10:42:40.426022: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-20 10:42:40.426064: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp44c2wt1k
2024-08-20 10:42:40.485395: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
